# Classification Predict - Climate Change

### Table of contents

### Problem identification

In [ ]:
# background and problem statement 

### What data do we have?

In [ ]:
# description of data

### Start experiment

In [ ]:
# Install necessary packages
# !pip install comet_ml

In [ ]:
# from comet_ml import Experiment

In [ ]:
# Setting the API key (saved as environment variable)
# experiment = Experiment(api_key="upOwchWrd7H1e6VEnWKW7PSvz", project_name="classification-predict", workspace="team-rm1")

### Import libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import SnowballStemmer
from nltk import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

### Import data

In [2]:
df_test = pd.read_csv('https://raw.githubusercontent.com/Amogelang20/RM1_classification_predict/dev/test.csv')
df_train = pd.read_csv('https://raw.githubusercontent.com/Amogelang20/RM1_classification_predict/dev/train.csv')

### Exploratory data analysis

In [ ]:
# look at the type of data that is present. 
# look at the types and number of columns present. 
# look at the y variable, the number of classes it has. 
# check for imbalance of data in the different classes of y variable. 
# check for missing values.
# visualise the data.


### Preprocessing

In [ ]:
#### CLEANING ####
# handle some of the unnecessary punctuation 
# upper/lower case
# change the slang words into something more meaningful to machine learning
# handle missing data and empty strings
# ect.


### Feature creation

In [ ]:
# Create some features from the given tweets,e.g length of tweet. Visualise these created features


### Split data into response and predictors

In [ ]:
#### clean_message = the cleaned text data ####
y = df_train['sentiment']
X = df_train['clean_message']


### Split data into training and validation sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Build pipelines to vectorize the data

In [ ]:
# Stem, tokenize and remove stopwords (all done within vectorization)
# Build a pipeline that vectorizes the text and creates classifiers for the different models 
# (logistic reg, SVM, Naive Bayes, Random Forest, Neural Nets)


In [ ]:
# Write class that has object that tokenizes text data AND stems the tokens
class StemAndTokenize:
    def __init__(self):
        self.ss = SnowballStemmer('english')
    def __call__(self, doc):
        return [self.ss.stem(t) for t in word_tokenize(doc)]


#### Logistic Regression

In [ ]:
# Create pipeline for Logistic Regression:
lr = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('lr', LogisticRegression())
              ])


#### Naïve Bayes

In [ ]:
# Create pipeline for Naïve Bayes:
nb = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('nb', MultinomialNB())
              ])


#### Linear SVM (Support Vector Machine)

In [ ]:
# Create pipeline for SVM:
Lsvm = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('Lsvm', LinearSVC())
              ])


#### Random Forest

In [ ]:
# Create pipeline for Random Forest:
rf = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('rf', RandomForestClassifier())
              ])


#### KNN (K Nearest Neighbors)

In [ ]:
# Create pipeline for KNN:
knn = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('knn', KNeighborsClassifier())
              ])


#### Neural Networks

In [ ]:
# Create pipeline for Neural Networks:
nn = Pipeline([('tfidf', TfidfVectorizer(tokenizer=StemAndTokenize())),
               ('nn', MLPClassifier())
              ])


### Hyperparameter optimization

In [ ]:
# optimize models by tuning parameters (GridSearch)


### Train models

#### Logistic Regression

In [ ]:
# Fitting the Logistic Regression model
lr.fit(X_train, y_train)


#### Naïve Bayes

In [ ]:
# Fitting the Naïve Bayes model
nb.fit(X_train, y_train)


#### Linear SVM (Support Vector Machine)

In [ ]:
# Fitting the SVM model
Lsvm.fit(X_train, y_train)


#### Random Forest

In [ ]:
# Fitting the Random Forest model
rf.fit(X_train, y_train)


#### KNN (K Nearest Neighbors)

In [ ]:
# Fitting the KNN model
knn.fit(X_train, y_train)


#### Neural Networks

In [ ]:
# Fitting the Neural Networks model
nn.fit(X_train, y_train)


### Evaluate model accuracy

In [ ]:
# confusion matrix
# classification report


### Make predictions

In [ ]:
# make predicts 


### Save the model

In [ ]:
# pickle model


### Log parameters

In [ ]:
# save parameters in variables to be logged to comet


### End experiment

### Conclusion